In [1]:
f = open('female.txt')
female_raw=f.readlines()
f1 = open('male.txt')
male_raw=f1.readlines()

Deleting possible repetitions and the '\n' in the end of each word

In [2]:
matches=set(female_raw).intersection(male_raw)

In [3]:
female=list(set(female_raw)-matches)
male=list(set(male_raw)-matches)

In [4]:
female=[i[:-1].lower() for i in female]
male=[i[:-1].lower() for i in male]

Making everything the same length. The lists are already shuffled by using the set function, so we dont need to worry about cutting out some of the prefixes

In [5]:
female=female[0:len(male)]

In [6]:
len(female)==len(male)

True

Making train set and test set

In [7]:
female_train=female[0:int(0.75*len(female))]
male_train=male[0:int(0.75*len(male))]

In [8]:
female_test=female[len(female_train):]
male_test=male[len(male_train):]

In [9]:
print(len(female_train), len(female_test), len(male_train), len(male_test))

1933 645 1933 645


In [10]:
max_len=len(max([max(female,key=len),max(male,key=len)], key=len))

In [13]:
import numpy as np
import string
alphabet=string.ascii_lowercase+'- '+'\''
def binarize_names(names, ind):
    binarized_names=[]
    for name in names:
        name_matrix = np.zeros((max_len, len(alphabet)), dtype='float32')
        for letter in name:
            name_matrix[name.index(letter), alphabet.index(letter)]=1.0
        binarized_names.append((name_matrix, ind))
    return binarized_names

In [14]:
import random
train_names=binarize_names(female_train, 1.0)+binarize_names(male_train, 0.0)
random.shuffle(train_names)
test_names=binarize_names(female_test, 1.0)+binarize_names(male_test, 0.0)
random.shuffle(test_names)

In [15]:
X_train=np.array([i[0] for i in train_names])
Y_train=np.array([i[1] for i in train_names])
X_test=np.array([i[0] for i in test_names])
Y_test=np.array([i[1] for i in test_names])

In [16]:
X_train.shape

(3866, 15, 29)

In [17]:
Y_train.shape

(3866,)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional

Using TensorFlow backend.


In [37]:
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(max_len, len(alphabet))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(512, return_sequences=False)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32, epochs=30, validation_split=0.1)

Train on 3479 samples, validate on 387 samples
Epoch 1/30
3479/3479 [==============================] - 142s - loss: 0.6398 - acc: 0.6444 - val_loss: 0.6251 - val_acc: 0.6718
Epoch 2/30
3479/3479 [==============================] - 141s - loss: 0.5899 - acc: 0.6887 - val_loss: 0.5925 - val_acc: 0.6951
Epoch 3/30
3479/3479 [==============================] - 160s - loss: 0.5620 - acc: 0.7166 - val_loss: 0.6200 - val_acc: 0.6796
Epoch 4/30
3479/3479 [==============================] - 151s - loss: 0.5500 - acc: 0.7261 - val_loss: 0.5994 - val_acc: 0.6770
Epoch 5/30
3479/3479 [==============================] - 148s - loss: 0.5395 - acc: 0.7321 - val_loss: 0.5670 - val_acc: 0.7028
Epoch 6/30
3479/3479 [==============================] - 152s - loss: 0.5267 - acc: 0.7459 - val_loss: 0.5516 - val_acc: 0.7132
Epoch 7/30
3479/3479 [==============================] - 156s - loss: 0.5228 - acc: 0.7358 - val_loss: 0.6047 - val_acc: 0.7028
Epoch 8/30
3479/3479 [==============================] - 151s - l

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 15, 512)           1110016   
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 512)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 1024)              4198400   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 1025      
_________________________________________________________________
activation_8 (Activation)    (None, 1)                 0         
Total params: 5,309,441
Trainable params: 5,309,441
Non-trainable params: 0
_________________________________________________________________


In [39]:
#dense=1, sigmoid, binary crossentropy, dropout=0.2/0.1, epochs=15,  batch_size=32, bilstm, 512 on both
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 16s    
[0.45310598621072695, 0.81007751928743466]


In [36]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2/0.1, epochs=30,  batch_size=32, bilstm, 512 on both layers
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 16s    
[0.6015921807104303, 0.80465116297551831]


In [34]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2/0.1, epochs=30,  batch_size=32, bilstm, 128 on hidden, 256 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 7s     
[0.4533819082171418, 0.80000000018482065]


In [30]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2, epochs=30,  batch_size=32, bilstm, 256 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 13s    
[0.57543140178502994, 0.7930232559987741]


In [27]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2, epochs=30,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 10s    
[0.54062903611234914, 0.80000000018482065]


In [24]:
#dense=1, sigmoid, binary crossentropy, dropout=0.2/0.1, epochs=15,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 7s     
[0.47647088433420937, 0.78372093041737878]


In [21]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2/0.1, epochs=5,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 10s    
[0.54307038266529406, 0.73023255804712461]


Old counts

In [38]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2/0.1, epochs=30,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 13s    
[0.61343265508496481, 0.71472868235536324]


In [32]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2/0.1, epochs=15,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 12s    
[0.5897029678026835, 0.68682170533394626]


In [29]:
#dense=2, sigmoid, sparse categorical crossentropy, dropout=0.2/0.1, epochs=5,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 13s    
[0.6073310375213623, 0.64806201559628629]


In [26]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.1, epochs=5,  batch_size=32, bilstm, 128 on hidden, 512 on first
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 14s    
[0.60862938775572661, 0.66356589138045796]


In [22]:
#dense=1, sigmoid, binary crossentropy, dropout=0.1, epochs=5,  batch_size=32, bilstm, 128 on both layers
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 5s     
[0.6287926376327988, 0.63643410843472148]


In [186]:
#dense=1, sigmoid, binary crossentropy, dropout=0.1, epochs=10,  batch_size=32, lstm, 512 on both layers
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 18s    
[0.64271246878675714, 0.62015503894451052]


In [181]:
#dense=1, sigmoid, binary crossentropy, dropout=0.5, epochs=10,  batch_size=16, lstm, 512 on both layers
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 26s    
[0.69337500601775881, 0.4999999999075897]


In [176]:
#dense=1, sigmoid, binary crossentropy, dropout=0.2, epochs=5,  batch_size=16, lstm, 512 on both layers
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 24s    
[0.6605583102204079, 0.62093023255813951]


In [168]:
#dense=2, softmax, sparse categorical crossentropy, dropout=0.2, epochs=5,  batch_size=16, lstm, 512 on both layers
score = model.evaluate(X_test, Y_test)

print(score)

1290/1290 [==============================] - 22s    
[0.65916604385819544, 0.60310077519379846]


In [12]:
np.zeros((3,4))

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])